In [ ]:
#!pip install -r requirements.txt

In [ ]:
import pandas as pd
from PIL import Image
from tqdm import tqdm
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pydot
import matplotlib.pyplot as plt

In [ ]:
images_path = '/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/deepfakes_detection/images'
root = '/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/deepfakes_detection'

In [ ]:
train = pd.read_csv(root + '/train.csv')
train.head()

In [ ]:
# Split the images between fake and not in two folders
# Real images
#for id in tqdm(train[train.label==0].image_id):
#    im = Image.open(f'{root}/images/{id}.jpg')
#    im.save(f'train/real_images/{id}.jpg')

In [ ]:
# Fake images
#for id in tqdm(train[train.label==1].image_id):
#    im = Image.open(f'{root}/images/{id}.jpg')
#    im.save(f'train/fake_images/{id}.jpg')

In [ ]:
test = pd.read_csv(root + '/test.csv')

In [ ]:
test.shape

In [ ]:
# Test
test = pd.read_csv(root + '/test.csv')

for id in tqdm(test.image_id):
    im = Image.open(f'{root}/images/{id}.jpg')
    im.save(f'/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/group_shared_workspace/hufflepuff_project/validation/valid_images/{id}.jpg')

In [ ]:
!pwd

In [ ]:
import pandas as pd
a = pd.read_csv('/home/jovyan/group4-deepfakes-detection/results/results.csv')
a.head()

In [ ]:
a.label = (a.label>.5)*1

In [ ]:
print(a.shape)
a.head()

In [ ]:
b.head(7)

In [ ]:
(a==b).all()

In [ ]:
b = pd.read_csv('sample_submission.csv', sep=';')

In [ ]:
print(b.shape)
b.head()

In [ ]:
a.to_csv('results2.csv')

In [ ]:
import os
os.chdir('/home/jovyan/group4-deepfakes-detection')

In [ ]:
!pwd

In [ ]:
import os
os.chdir('/home/jovyan/group4-deepfakes-detection')

In [ ]:
from tools.deepfake_reco import *

with open('/home/jovyan/group4-deepfakes-detection/config.yaml', 'rb') as f:
    conf = yaml.safe_load(f.read()) 

image_size = tuple(conf.get('IMAGE_SIZE'))
batch_size = conf.get('BATCH_SIZE')
n_epochs = conf.get('N_EPOCHS')
train_test_path = conf.get('TRAIN_TEST_PATH')
eval_path = conf.get('EVAL_PATH')
eval_id_path = conf.get('EVAL_ID_PATH')

# Pipeline start
# Dataset
train_ds, test_ds, eval_ds = data_generation(image_size, batch_size, train_test_path, eval_path)

In [ ]:
eval_ds

In [ ]:
folder_path = '/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/group_shared_workspace/hufflepuff_project/test/'
test_ds = tfds.folder_dataset.ImageFolder(folder_path, *, (180, 180))

In [ ]:
image_size = (180, 180)
batch_size = 32
train_path = '/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/group_shared_workspace/hufflepuff_project/train'

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    validation_split=0.2,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
valid_path = '/home/jovyan/hfactory_magic_folders/tooling_for_the_data_scientist/group_shared_workspace/hufflepuff_project/val'
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_path,
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
import yaml

with open('/home/jovyan/group4-deepfakes-detection/config.yaml', 'rb') as f:
    conf = yaml.safe_load(f.read()) 
conf.get('TRAIN_TEST_PATH')

In [ ]:
conf.get('TRAIN_TEST_PATH')

In [ ]:
val_ds

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

In [ ]:
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    # Image augmentation block
    #x = data_augmentation(inputs)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(32, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [128, 256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)

In [ ]:
target = pd.read_csv('/home/jovyan/group4-deepfakes-detection/sample_submission.csv')
target.head()

In [ ]:
model = make_model(input_shape=image_size + (3,), num_classes=2)
#keras.utils.plot_model(model, show_shapes=True)

In [ ]:
epochs = 1

callbacks = [
    keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5"),
]
model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds,
)

In [ ]:
type(model)